In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

In [2]:
!pip install -r requirements.txt

    100% |████████████████████████████████| 92.6MB 521kB/s ta 0:00:01    29% |█████████▌                      | 27.6MB 467kB/s eta 0:02:20    97% |███████████████████████████████ | 90.0MB 415kB/s eta 0:00:07
    100% |████████████████████████████████| 317kB 410kB/s ta 0:00:01
    100% |████████████████████████████████| 368kB 414kB/s ta 0:00:01
    100% |████████████████████████████████| 3.2MB 528kB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 533kB/s a 0:00:01
    100% |████████████████████████████████| 10.8MB 526kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 188kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 920kB/s ta 0:00:011
    100% |████████████████████████████████| 1.2MB 511kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 824kB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 706kB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 547kB/s ta 0:00:01
  Stored in directory: /root/.cache/

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 1s 174us/step - loss: 0.4793 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.4268 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 194us/step - loss: 0.4213 - acc: 0.8024
Epoch 4/100
8000/8000 [==============================] - 2s 201us/step - loss: 0.4177 - acc: 0.8229
Epoch 5/100
8000/8000 [==============================] - 2s 211us/step - loss: 0.4164 - acc: 0.8262
Epoch 6/100
8000/8000 [==============================] - 2s 304us/step - loss: 0.4145 - acc: 0.8294
Epoch 7/100
8000/8000 [==============================] - 2s 259us/step - loss: 0.4130 - acc: 0.8324
Epoch 8/100
8000/8000 [==============================] - 2s 274us/step - loss: 0.4114 - acc: 0.8312
Epoch 9/100
8000/8000 [==============================] - 1s 185us/step - loss: 0.4097 - acc: 0.8346
Epoch 10/100
8000/8000 [============================

8000/8000 [==============================] - 2s 214us/step - loss: 0.4011 - acc: 0.8339
Epoch 80/100
8000/8000 [==============================] - 2s 294us/step - loss: 0.4005 - acc: 0.8356
Epoch 81/100
8000/8000 [==============================] - 1s 174us/step - loss: 0.4006 - acc: 0.8344
Epoch 82/100
8000/8000 [==============================] - 2s 231us/step - loss: 0.4004 - acc: 0.8344
Epoch 83/100
8000/8000 [==============================] - 2s 250us/step - loss: 0.4009 - acc: 0.8329
Epoch 84/100
8000/8000 [==============================] - 2s 228us/step - loss: 0.4003 - acc: 0.8344
Epoch 85/100
8000/8000 [==============================] - 1s 147us/step - loss: 0.4004 - acc: 0.8347
Epoch 86/100
8000/8000 [==============================] - 1s 140us/step - loss: 0.4004 - acc: 0.8336
Epoch 87/100
8000/8000 [==============================] - 2s 230us/step - loss: 0.4008 - acc: 0.8330
Epoch 88/100
8000/8000 [==============================] - 2s 305us/step - loss: 0.4005 - acc: 0.8342
Epo

In [11]:

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)